# Sharing data between threads

## Potential problem

The threads launched by the same program see the same memory area. They can possibly interact at the same time on the same data and produce undesirable effects. In the following example the functions `compute` and `print` are executed within two different threads. We see that `print` can interfere during the execution of `compute` between the increment of the number and the calculation of the square, which can produce an erroneous display.

In [ ]:
%%file tmp.mutex.h

#include <iostream>
#include <chrono>
#include <thread>
#include <cstdlib>
#include <cassert>
#include <mutex>

using namespace std::chrono_literals ;

In [ ]:
%%file tmp.mutex-calculator.h

class Calculator
{
 public :
   void compute()
    {
     m_number++ ;
     std::this_thread::sleep_for(900us) ;
     m_square = m_number*m_number ;
    }
   void print()
    {  std::cout<<m_number<<" squared "<<m_square<<std::endl ; }
 private :
   int m_number{}, m_square{} ;
} ;

In [ ]:
%%file tmp.mutex-repeaters.h

void repeat_print( int nb, Calculator & c )
{
 for ( int i=0 ; i<nb ; ++i )
  {
   c.print() ;
   std::this_thread::sleep_for(1ms) ;
  }
}
void repeat_compute( int nb, Calculator & c )
{
 for ( int i=0 ; i<nb ; ++i )
  {
   c.compute() ;
   std::this_thread::sleep_for(100us) ;
  }
}

In [ ]:
%%file tmp.mutex.cpp

#include "tmp.mutex.h"
#include "tmp.mutex-calculator.h"
#include "tmp.mutex-repeaters.h"

int main( int argc, char * argv[] )
{
  assert(argc==2) ;
  int nb = atoi(argv[1]) ;
  Calculator c ;
  std::thread t1(repeat_print,nb,std::ref(c)) ;
  std::thread t2(repeat_compute,nb,std::ref(c)) ;
  t1.join() ; t2.join() ;
 }

In [ ]:
%%file tmp.mutex.sh
echo

rm -f tmp.mutex.exe \
&& g++ -std=c++17 -lpthread tmp.mutex.cpp -o tmp.mutex.exe\
&& ./tmp.mutex.exe $*

echo

In [ ]:
! bash -l tmp.mutex.sh 10

## Locking a portion of code

To avoid the above situation, C ++ 11 offers the notion of a lock via the `std::mutex` class, which guarantees that a certain portion of code is only executed by one process at a time. Below, a correction for the class `Calculator`.

In [ ]:
%%file tmp.mutex-calculator.h

class Calculator
{
 public :
   void compute()
    {
     m_mtx.lock() ; // lock acquisition
     m_number++ ;
     std::this_thread::sleep_for(900us) ;
     m_square = m_number*m_number ;
     m_mtx.unlock() ; // lock release
    }
   void print()
    {
     m_mtx.lock() ; // lock acquisition
     std::cout<<m_number<<" squared "<<m_square<<std::endl ;
     m_mtx.unlock() ; // lock release
    }
 private :
   int m_number{} ,m_square{} ;
   std::mutex m_mtx ; // can only be acquired by one thread at a time
} ;

In [ ]:
! bash -l tmp.mutex.sh 10

**BEWARE**: the mutex used by multiple threads must be the same single object. It can be passed by reference to processes and their functions via an `std::ref`, or it can be enclosed in an object passed in this way. It can also be a global, static, or heap-allocated object with a `new`. In all cases, for the locking to be effective, the different processes involved must view the same object.

## Protection against exceptions

This code is not completely protected against the occurrence of an exception, which could leave locks blocked indefinitely. As with all resources, we can improve this code with an object responsible for acquiring a lock on construction and releasing it on destruction. To manage `std::mutex` in this way, C++11 introduced `std::lock_guard`. C++17 added **`std::scoped_lock`** which can lock several mutexes simultaneously.

In [ ]:
%%file tmp.mutex-calculator.h

class Calculator
{
 public :
   void compute()
    {
     std::scoped_lock<std::mutex> lg(m_mtx) ;
     m_number++ ;
     std::this_thread::sleep_for(500us) ;
     m_square = m_number*m_number ;
    }
   void print()
    {
     std::scoped_lock<std::mutex> lg(m_mtx) ;
     std::cout<<m_number<<" squared "<<m_square<<std::endl ;
    }
 private :
   int m_number{} ,m_square{} ;
   std::mutex m_mtx ; // can only be acquired by one thread at a time
} ;

In [ ]:
! bash -l tmp.mutex.sh 10

### Additional remarks

- `mutex::try_lock()` allows to check whether or not the mutex is locked.
- `unique_lock` is a "movable" variant of `lock_guard`.
- `recursive_mutex`: can be acquired more than once (incrementation of a counter) and freed as many times (decrementation of the counter) for recursive approaches.
- `timed_mutex`: the `try_lock_for` and `try_lock_until` methods allow to set a time limit for lock acquisition.
- `recursive_timed_mutex`: combines the properties of `recursive_mutex` and `timed_mutex`.

# Questions ?

# Exercise

In the program below, we added a display within the `complexes_pow` function, which is executed by threads. Normally, when running the program several times, you will notice anomalies in the display. Indeed, like any variable seen and shared by several threads, `std::cout` can malfunction when it is used at the same time by several threads. To solve this, use an `std::mutex` to lock the portion of code that is displaying.

In [ ]:
%%file tmp.mutex.cpp

#include <complex>
#include <vector>
#include <iostream>
#include <cassert>
#include <cmath>
#include <thread>
#include <mutex>

using Real = double ;
using Complex = std::complex<Real> ;
using Complexes = std::vector<Complex> ;

// random unitary complexes
void generate( Complexes & cs )
 {
  srand(1) ;
  for ( auto & c : cs )
   { 
    Real angle {rand()/(Real(RAND_MAX)+1)*2.0*M_PI} ;
    c = Complex{std::cos(angle),std::sin(angle)} ;
   }
 }

// compute a slice of xs^degree and store it into ys
// xs.size() must be a multiple of nb_slices
void complexes_pow
 ( std::size_t num_slice, std::size_t nb_slices,
   Complexes const & xs, int degree, Complexes & ys )
 {
  assert((xs.size()%nb_slices)==0) ;  
  auto slice_size {xs.size()/nb_slices} ;
  auto min {num_slice*slice_size} ;
  auto max {(num_slice+1)*slice_size} ;
  std::cout<<"complexes_pow "<<num_slice<<" min : "<<min<<std::endl ;     
  std::cout<<"complexes_pow "<<num_slice<<" max : "<<max<<std::endl ; 
  for ( auto i {min} ; i<max ; ++i )
   {
    ys[i] = Complex{1.,0.} ;
    for ( int d=0 ; d<degree ; ++d )
     { ys[i] *= xs[i] ; }
   }
 }

// display the angle of the complexes global product
void postprocess( Complexes const & cs )
 {
  Complex prod(1.,0.) ;
  for( auto c : cs ) { prod *= c ; }
  double angle {atan2(prod.imag(),prod.real())} ;
  std::cout<<"result = "<<static_cast<int>(angle/2./M_PI*360.)<<"\n" ;
 }

// main program
int main ( int argc, char * argv[] )
 {
  assert(argc==4) ;
  std::size_t nbtasks {std::stoul(argv[1])} ;
  std::size_t dim {std::stoul(argv[2])} ;
  int degree {std::stoi(argv[3])} ;

  // prepare input
  Complexes input(dim) ;
  generate(input) ;
   
  // compute
  Complexes output(dim) ;
  std::size_t numtask ;
  std::vector<std::thread> workers ;
  for ( numtask = 0 ; numtask<nbtasks ; ++numtask )
   { workers.emplace_back(complexes_pow,numtask,nbtasks,std::ref(input),degree,std::ref(output)) ; }
  for ( auto & worker : workers )
   { worker.join() ; }
  
  // post-process
  postprocess(output) ;
 }


In [ ]:
%%file tmp.mutex.sh
echo

rm -f tmp.mutex.exe \
&& g++ -std=c++17 -lpthread tmp.mutex.cpp -o tmp.mutex.exe\
&& ./tmp.mutex.exe $*

echo

In [ ]:
! bash -l tmp.mutex.sh 2 2 3

In [ ]:
! bash -l tmp.mutex.sh 4 1024 100000

© *CNRS 2024*  
*This document was created by David Chamont and translated by Olga Abramkina. It is available under the [License Creative Commons - Attribution - No commercial use - Shared under the conditions 4.0 International](http://creativecommons.org/licenses/by-nc-sa/4.0/)*